# Проект Автодополнение текстов

Вы работаете в соцсетевом приложении, где пользователи постят короткие тексты. В продукте стоит задача — добавить возможность автодополнения текстов. Разработчики просят вас создать модель, которую можно запускать на мобильных устройствах. Для смартфонов есть значительные требования по оперативной памяти и скорости работы, так что важна легковесность модели. 

## Постановка задачи
**Создать нейросеть, которая на основе начала фразы предсказывает её продолжение**.

Поэтапное описание задачи:
1. Взять датасет от разработчиков, очистить его, подготовить для обучения модели.
1. Реализовать и обучить модель на основе рекуррентных нейронных сетей.
1. Замерить качество разработанной и обученной модели.
1. Взять более «тяжёлую» предобученную модель из Transformers и замерить её качество.
1. Проанализировать результаты и **дать рекомендации** разработчикам: стоит ли **использовать лёгкую модель или** лучше постараться поработать с ограничениями по памяти и **использовать большую предобученную**.

## Критерии успеха
- Используемые метрики качества: ROUGE-1 и ROUGE-2
- Минимальных порогов качества модели в задаче не задано


## Описание данных

датасет с короткими постами sentiment140

https://code.s3.yandex.net/deep-learning/tweets.txt

# Этап 0. Подготовка окружения

В задании требовалось весь код убрать в директории. Код доступен тут:

https://github.com/evgeny-ydsc/DLE-NLP

In [1]:
#!pip install -r requirements_sprint_2_project.txt

In [2]:
import evaluate
from src.data_utils import download_dataset
from src.next_token_dataset import prepare_loaders
from src.lstm_model import LSTMGenerator
from src.lstm_train_eval import ModelTrainer

c:\ProgramData\anaconda3\envs\practicumNLP-py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Этап 1. Сбор и подготовка данных

👷🚧🚧🚧🚧🚧 перед загрузкой на GPU заменить 200 на 20000

In [3]:
raw_data_path = download_dataset()
train_loader, val_loader, test_loader, tokenizer, val_texts, test_texts = \
    prepare_loaders (raw_data_path, max_texts_count=50)

# Этап 2. Реализация рекуррентной сети


In [4]:
model = LSTMGenerator(tokenizer)

# Этап 3. Тренировка модели

In [5]:
trainer = ModelTrainer (model, tokenizer)
trainer.train(train_loader, val_loader, val_texts)

100%|██████████| 10/10 [00:00<00:00, 14.18it/s]


Epoch 1 | Train Loss: 10.295 | Val Loss: 10.200 | Val Accuracy: 0.00%| rouge1: 0.000 | rouge2: 0.000


100%|██████████| 10/10 [00:00<00:00, 16.45it/s]


Epoch 2 | Train Loss: 9.528 | Val Loss: 8.981 | Val Accuracy: 4.44%| rouge1: 0.000 | rouge2: 0.000


100%|██████████| 10/10 [00:00<00:00, 14.21it/s]


Epoch 3 | Train Loss: 6.531 | Val Loss: 8.572 | Val Accuracy: 6.67%| rouge1: 0.000 | rouge2: 0.000


In [6]:
rouge = evaluate.load("rouge")
for line in test_texts[:3]:
    l = len(line)
    i = int(l*0.75)
    start = line [:i]
    finish = line [i:]
    predicted_finish = model.generate_output_text(start, l-i)
    results = rouge.compute(predictions=[predicted_finish], references=[finish])
    print (f"оригинальная строка: {line}")
    print (f"оригинальное начало: {start}")
    print (f"оригинальный конец : {finish}")
    print (f"предсказанный конец: {predicted_finish}")
    print (f"rouge1={results['rouge1']:.3f}, rouge2={results['rouge2']:.3f}\n")
    

оригинальная строка: really dont feel like getting up today but got to study to for tomorrows practical exam
оригинальное начало: really dont feel like getting up today but got to study to for to
оригинальный конец : morrows practical exam
предсказанный конец: iiiiiiiiiiiiiiiiiiiiii
rouge1=0.000, rouge2=0.000

оригинальная строка: oh manwas ironing jeancjumbes fave top to wear to a meeting burnt it
оригинальное начало: oh manwas ironing jeancjumbes fave top to wear to a
оригинальный конец :  meeting burnt it
предсказанный конец: iiiiiiiiiiiiiiiii
rouge1=0.000, rouge2=0.000

оригинальная строка: where did u move to i thought u were already in sd hmmm random u found me glad to hear yer doing well
оригинальное начало: where did u move to i thought u were already in sd hmmm random u found me g
оригинальный конец : lad to hear yer doing well
предсказанный конец: iiiiiiiiiiiiiiiiiiiiiiiiii
rouge1=0.000, rouge2=0.000



👷🚨[SOS] тут, очевидно, что в функции generate_output_text (и/или в forward) я делаю что-то не так - прошу помочь разобраться.

# Этап 4. Использование предобученного трансформера

# Этап 5. Формулирование выводов


- В соответствии с поставленной задачей(см. первый раздел)
    - была проведена подготовка данных и их разведочный анализ, в результате которых:
        - были исправлены имена полей, исправлены типы данных, удалены малополезные признаки (в т. ч. с учётом мультиколлинеарности), заполнены пропуски, удалены явные и неявные дубликаты, ...
    - была натренирована модель, предсказывающая ...
        - выбрана наиболее оптимальная модель - **...**, показавшая наилучшие результаты:
            - качество предсказания: ...
            - это удовлетворяет заданный критерий качества: ...